In [72]:
from lxml import html, etree

import requests
import time

url_form = 'http://forum.spore.com/jforum/posts/list/{0}/{1}.page'
page_cache = {}

In [92]:
def make_tree(offset, topic_id):
    url = url_form.format(offset, topic_id)
    
    if topic_id in page_cache and offset//15 in page_cache[topic_id]:
        page = page_cache[topic_id][offset // 15]
    else:
        page = requests.get(url)
            
    if offset is not 0:
        page_cache[topic_id].append( page )
    else:
        page_cache[topic_id] = [page]
    
    return html.fromstring(page.content)

In [109]:
def scrape_topic(topic_id):
    tree = make_tree(0, topic_id)
    
    page_numbers = [el.text_content() for el in tree.cssselect('.pagination a')]
    # The last two pagination buttons are a "next page" arrow and "Go" button,
    # so the actual last page number is third from the end.
    # If the topic only has one page, there's no pagination buttons at all.
    page_count = int(page_numbers[-3]) if page_numbers else 1

    def scrape(offset, topic_id):
        tree = make_tree(offset, topic_id) # TODO: rate limiter
        print(offset//15 + 1, '/', page_count)
        
        usernames = [el.text_content() for el in tree.cssselect('.genmed')]
        # if not usernames: return None  # if no usernames, this is a void topic. bail out!

        dates = [el.text_content() for el in tree.cssselect('.date')]
        post_ids = [el.cssselect('a')[1].get('name') for el in tree.cssselect('.postinfo')]
        for i, post_id in enumerate(post_ids):
            
            root = tree.cssselect('#post_text_{}'.format(post_id))[0]
            # for el in root.iter('blockquote'):
                # if el.text: print(usernames[i], el.text_content().strip())
                # if el.text is not None: el.text = '> ' + el.text # codecs.escape_decode(el.content)[0].decode()
                    
            content = root.cssselect('.postbody')
            for el in content[0].iter('*'):
                if el.text: el.text = el.text.strip()
                if el.tail: el.tail = el.tail.strip()
                    
            attach = root.cssselect('.gensmall')
                    
            yield {
                'id': post_id,
                'user': usernames[i],
                'date': dates[i].strip(),
                'content': etree.tostring(content[0], encoding='utf-8'),
                'edits': etree.tostring(attach[0], encoding='utf-8') if attach else '',
            }

    ret = [scrape(j*15, topic_id) for j in range(0, page_count)] # make list of generators
    
    return [post for page in ret for post in page] # spool out all posts

In [164]:
tsv_cols = ['id', 'user', 'date', 'content', 'edits']
tsv_headers = '\t'.join(tsv_cols)

# FEED scraper in background
# TODO: collect changes in title of thread from its replies

topics = [64027, 30084, 17489, 55189, 43567, 55225, 58125, 58991, 74506,
    43568, 20495, 17610, 74664, 72414, 58559, 14795, 72634] # OP = ChaosHarbinger

# [41538, 45708, 57096, 56446, 49553, 45652, 52134, 64863, 78116, 55544, 52386,
# 47166, 45328, 68928, 46926, 52686, 53024, 41427, 45383, 47032, 48660, 49839, 52783, 53762, 65289] # OP = eSCSi

for thread in topics:
    with open('./{}.tsv'.format(thread), 'wb') as tsv:
        tsv.write((tsv_headers + '\n').encode('utf-8'))
        topic = scrape_topic(thread)
        
        for post in topic:
            for col in tsv_cols:
                res = post[col] if isinstance(post[col], bytes) else post[col].encode('utf-8')
                tsv.write(res)
                tsv.write(('\t' if col is not tsv_cols[-1] else '\n').encode('utf-8'))
                
            # tsv_line = '\t'.join([str(post[col]) for col in tsv_cols])            
        time.sleep(1)  # to avoid overwhelming the poor server :(

1 / 193
2 / 193
3 / 193
4 / 193
5 / 193
6 / 193
7 / 193
8 / 193
9 / 193
10 / 193
11 / 193
12 / 193
13 / 193
14 / 193
15 / 193
16 / 193
17 / 193
18 / 193
19 / 193
20 / 193
21 / 193
22 / 193
23 / 193
24 / 193
25 / 193
26 / 193
27 / 193
28 / 193
29 / 193
30 / 193
31 / 193
32 / 193
33 / 193
34 / 193
35 / 193
36 / 193
37 / 193
38 / 193
39 / 193
40 / 193
41 / 193
42 / 193
43 / 193
44 / 193
45 / 193
46 / 193
47 / 193
48 / 193
49 / 193
50 / 193
51 / 193
52 / 193
53 / 193
54 / 193
55 / 193
56 / 193
57 / 193
58 / 193
59 / 193
60 / 193
61 / 193
62 / 193
63 / 193
64 / 193
65 / 193
66 / 193
67 / 193
68 / 193
69 / 193
70 / 193
71 / 193
72 / 193
73 / 193
74 / 193
75 / 193
76 / 193
77 / 193
78 / 193
79 / 193
80 / 193
81 / 193
82 / 193
83 / 193
84 / 193
85 / 193
86 / 193
87 / 193
88 / 193
89 / 193
90 / 193
91 / 193
92 / 193
93 / 193
94 / 193
95 / 193
96 / 193
97 / 193
98 / 193
99 / 193
100 / 193
101 / 193
102 / 193
103 / 193
104 / 193
105 / 193
106 / 193
107 / 193
108 / 193
109 / 193
110 / 193
111 / 19

#### roundtrip of serialization

In [148]:
import csv

# 20495, 22  # taxonomization
# 45652, 1   # phylum reboot

# ret = scrape_topic(45652)
with open('./{}.tsv'.format(topics[0]), 'r') as f:
    ret = list( csv.DictReader(f, delimiter='\t') )

In [149]:
for i,u in enumerate(ret):
    print( i, u['user'], u['date'] )

0 ChaosHarbinger 02/12/2009 01:20:04
1 Kerslam 02/12/2009 01:40:58
2 cpduhe 02/12/2009 01:44:06
3 Tutthoth 02/12/2009 01:53:24
4 wvbobster 02/12/2009 01:57:52
5 ChaosHarbinger 02/12/2009 02:10:36
6 metlwing 02/12/2009 03:42:20
7 wvbobster 02/12/2009 11:20:13
8 Ryuujin 02/12/2009 11:33:58
9 ESPEE 02/12/2009 15:56:40
10 cpduhe 02/12/2009 22:27:01
11 Voter 02/12/2009 22:47:36
12 Wii12 02/13/2009 01:26:57
13 Archereon 02/13/2009 01:40:42
14 Kerslam 02/13/2009 01:53:30
15 Vilageidiotx 02/13/2009 02:02:07
16 Ryuujin 02/13/2009 17:15:52
17 Vilageidiotx 02/13/2009 17:30:13
18 Cheeso33 02/13/2009 18:43:11
19 Voter 02/13/2009 22:39:17
20 ChaosHarbinger 02/14/2009 00:09:30
21 metlwing 02/14/2009 03:24:50
22 ChaosHarbinger 02/15/2009 21:39:04
23 Falthron 02/16/2009 00:20:58
24 Ryuujin 02/16/2009 02:12:52
25 ChaosHarbinger 02/16/2009 02:24:59
26 visitjoan 02/16/2009 03:00:04
27 Kerslam 02/16/2009 03:20:32
28 Falthron 02/16/2009 03:50:56
29 Voter 02/16/2009 04:47:27
30 ChaosHarbinger 02/16/2009 12:0

In [162]:
from IPython.core.display import HTML

# strip markup
print( '\n'.join([(el.text or '') + ' ' + (el.tail or '')
                  for el in etree.fromstring(ret[22]['content']).iter('*')]) )

# TODO: trim tabs but not spaces.
# HTML(ret[20]['content'])

 
 
 
Voter's Vampr Nocturi
 
 
Kingdom: Animalia
 
Sub-kingdom: Eukaryota
 
Infra-kingdom: Bilateria
 
Micro-kingdom: Deuterostomia
 
Phylum: Chordata
 
Sub-phylum: Craniata
 
Infra-phylum: Vertebrata
 
Micro-phylum: Gnathostomata
 
Classifying the Vampr Nocturi at this stage is fairly simple, given that they possess all the obvious traits of a highly developed organism, such as bilateral symmetry, a skull and well-formed backbone and neural system, a mouth with jaws, and other such anatomical features that marks them out as being particularly fine examples of macro-evolution. 
 
 
Class: Eunychophora
 
Sub-class: Octapoda
 
Infra-Class: Synapsida
 
Parv-class: Mammaliaformes
 
Owing to the number of the limbs possessed by the Vampr Nocturi, that being eight in number, they belong to the "eight-legged" sub-class of animals, given that in relation to non-insects, wings are considered more as specialised legs than a seperate physical component, which is supported by the fact that whilst